# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [108]:
%matplotlib inline
import pandas as pd
import pylab
pylab.rcParams['figure.figsize'] = (20.0, 10.0)
from xml.etree import ElementTree as ET

In [109]:
from IPython.core.display import HTML
HTML('<style>{}</style>'.format( open('style.css').read() ) )

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [6]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [7]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [193]:
document = ET.parse( './data/mondial_database.xml' )

data = {
    'countries': [],
    'cities': [],
    'ethnicity':[]
}

for country in document.iterfind('country'):
    country_info = {field.tag: field.text for field in country.getchildren() if len(field.attrib)==0}       
    country_info['population'] = [p.text for p in city.getiterator('population')]

    for city in country.getiterator('city'):
        
        data['cities'].append({
            'country': country_info['name'],
            'name': city.find('name').text,
            'population': [p.text for p in city.getiterator('population')]
        })
        
        
        
    for ethnic_group in country.getiterator('ethnicgroup'):
        
        data['ethnicity'].append({
            'name': ethnic_group.text,
            'percentage': ethnic_group.attrib['percentage'],
            'country_population': country_info['population'],
            'country': country_info['name']
        })
        
    data['countries'].append(country_info)
    
        

df = {}
for category, d in data.items():
    df[category] = pd.DataFrame(d).set_index('name')  

df['ethnicity'].country_population = df['ethnicity'].country_population.apply(lambda l: l[-1] if len(l) > 0 else 0).astype(int)
df['ethnicity'].percentage = df['ethnicity'].percentage.astype(float)
df['cities'].population = df['cities'].population.apply(lambda l: l[-1] if len(l) > 0 else 0).astype(int)
df['countries'].infant_mortality = df['countries'].infant_mortality.astype(float)

In [190]:
df['countries'].infant_mortality.sort_values().head(10).reset_index()#[::-1].plot.barh(fontsize=20)

,name,infant_mortality
0,Monaco,1.81
1,Japan,2.13
2,Bermuda,2.48
3,Norway,2.48
4,Singapore,2.53
5,Sweden,2.60
6,Czech Republic,2.63
7,Hong Kong,2.73
8,Macao,3.13
9,Iceland,3.15


In [191]:
df['cities'].population.sort_values(ascending=False).reset_index().head(10)#[::-1].plot.barh(fontsize=20)

,name,population
0,Shanghai,22315474
1,Istanbul,13710512
2,Mumbai,12442373
3,Moskva,11979529
4,Beijing,11716620
5,São Paulo,11152344
6,Tianjin,11090314
7,Guangzhou,11071424
8,Delhi,11034555
9,Shenzhen,10358381


In [208]:
df['ethnicity']['population'] = df['ethnicity'].country_population * (df['ethnicity'].percentage / 100.)
df['ethnicity'].reset_index().groupby(['name']).population.sum().sort_values(ascending=False).head(10).reset_index()

,name,population
0,Chinese,7.673916e+06
1,Black,6.444499e+06
2,African,6.289224e+06
3,Sinhalese,3.756758e+06
4,European,3.289928e+06
5,White,3.152595e+06
6,Armenian,2.310014e+06
7,Arab,2.044082e+06
8,Mestizo,1.855165e+06
9,Latvian,1.089227e+06


In [ ]:
!git add . 
!git commit -m 'practice importing xml data into pandas'
!git push origin 3.3